# Paper: Ocean mixing in a shelf seas driven by energetic nonlinear internal waves

Prepared for Journal of Geophysical Research: Oceans \
Notebook author: Chris A Whiwtwell \
Contact: chris.a.whitwell@outlook.com \
Date: 27/01/2023

This notebook contains the analysis to estime the diapycnal diffusivity, K, from the model described by Ivey et al. (2018) and using the method first described in Jones et al. (2020) for one site (Example here is for 200m deep mooring).\
All imported modules are pip installable.

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd

import statsmodels.tsa.stattools as ts
from scipy import stats
from scipy import signal
import wmtsa.modwt

import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
def mean_fn(x,axis=0,thresh=15):
    if np.count_nonzero(~np.isnan(x)) >= thresh:
        return np.nanmean(x,axis=axis)
    else:
        return np.nan

def geometric_mean_fn(x,axis=0,thresh=15):
    if np.count_nonzero(~np.isnan(x)) >= thresh:
        return 10**(np.nanmean(np.log10(x),axis=axis))
    else:
        return np.nan    

## Used to do a runs test for stationarity
def prep(x,axis=0):
    return np.nanmean(np.square(x),axis=axis)

def runs(x,axis=0):
    x_m = np.nanmedian(x,axis=axis)
    x_p = np.roll(x,1,axis=axis)
    
    runs =  np.nansum((x>=x_m[:,:,None])*(x_p<x_m[:,:,None]) + (x < x_m[:,:,None])*(x_p >= x_m[:,:,None]),axis=axis,dtype='int64')
    n1 = np.nansum(x>= x_m[:,:,None],axis=axis,dtype='int64')
    n2 = x.shape[axis[0]]-n1  
    runs_exp = ((2*n1*n2)/(n1+n2))+1
    stan_dev = np.sqrt((2*n1*n2*(2*n1*n2-n1-n2))/(((n1+n2)**2)*(n1+n2-1)))
    z = (runs-runs_exp)/stan_dev 
    return abs(z)

In [ ]:
ctd_path =  r"../Data/SBE37SM-RS232_03711063_2019_04_24.cnv"
SBE56_path = r"../Data/RowleyShoals_Gridded_Mooring_T_20sec.nc"
ADCP_path =  r"../Data/RowleyShoals2019_RDIADCPData_float64.nc"

In [ ]:
names = ['time','depth','temperature','conductivity','density','C1','C2','C3']
df_CTD =  pd.read_csv(ctd_path,names=names,skiprows=304,delim_whitespace=True)
mask = np.argwhere(df_CTD['depth'].values > 180)
slope, intercept, r, p, se = stats.linregress(df_CTD['temperature'].values[mask].flatten(), df_CTD['density'].values[mask].flatten())
print(f' rho = {slope:0.3f} theta + {intercept:0.3f}\n r =  {r} \n p = {p} \n SE = {se}')

In [ ]:
tslyce = slice('2019-03-07 00:00','2019-04-06 00:00')
ds_T =  xr.open_dataset(SBE56_path,group='T200').sel(time=tslyce).sel(depth=slice(8,160))
# paths
ds_ADCP = xr.open_dataset(ADCP_path,group='T200_RDI_75kHz_LR_24613').sel(time=tslyce).sel(distance=slice(0,170))
ds_ADCP = ds_ADCP.rename({'distance':'depth'})
ds_ADCP = ds_ADCP[['u','v']].interpolate_na(dim='time',limit=1)
ds_ADCP =  ds_ADCP.rolling(time=3,min_periods=1).mean()

depths = ds_ADCP.depth.values
depth_dim = len(ds_ADCP.depth.values)
time_dim = len(ds_ADCP.time.values)

## Estimate shear analytically from Chebyshev smoothed velocity data 
cheby_order_shear = 20
S =  np.empty((depth_dim,time_dim))

for t,time in enumerate(ds_ADCP.time.values):
    subt = ds_ADCP.isel(time=t)
    Efit = np.polynomial.chebyshev.Chebyshev.fit(subt.depth.values,subt['u'].values,cheby_order_shear).deriv(m=1)
    Nfit = np.polynomial.chebyshev.Chebyshev.fit(subt.depth.values,subt['v'].values,cheby_order_shear).deriv(m=1)
    S[:,t] = np.sqrt((Efit(depths)**2) + (Nfit(depths)**2))

S_int = xr.DataArray(S,dims=['depth','time'],coords={'depth':ds_ADCP.depth.values,'time':ds_ADCP.time.values})

ds_T['Sb'] =  S_int.interp_like(ds_T['Temperature'],method='linear')
ds_T['Sb'] = ds_T['Sb'].interpolate_na(dim='depth',method='nearest',fill_value='extrapolate')

ds_T.attrs['slope'] = slope
ds_T.attrs['intercept'] = intercept

In [ ]:
## Now do a wavelet transform on all SBE56 data
T_wavelet_coeff = [] ## rotated
T_smooth = []
T_details = []

for d,depth in enumerate(ds_T.depth.values):
    print(f'Performing MODWT for {depth} thermistor, progress: {d*100/len(ds_T.depth.values)}')
    thermistor =  ds_T['Temperature'].sel(depth=depth).values
    ### T
    w,v = wmtsa.modwt.modwt(thermistor,wtf='la16')
    J0 = w.info['J0']
    # Hard thresholding
    sigma = 0.001
    N = len(thermistor)
    js = np.arange(1,J0+1)
    thresh = np.ones_like(w)*np.sqrt(2*(sigma**2)*np.log(N)/(2**js))[:,None]
    w[abs(w) < thresh] = 0
    wr,vr = wmtsa.modwt.cir_shift(w,v,subtract_mean_VJ0t=True)
    T_wavelet_coeff.append(np.asarray(wr))
    T_smooth.append(np.asarray(wmtsa.modwt.imodwt_smooth(v)))
    T_details.append(np.asarray(wmtsa.modwt.imodwt_details(w)))
    print(f'Completed MODWT for {depth} thermistor')

js = np.arange(1,J0+1)
j_low = 20*(2**js) # 2**js if using SBE56s
j_high = 20*(2**(js+1))# 2**(js+1) if using SBE56s

ds_T['js'] =  xr.DataArray(js,dims=['js'],attrs={'long_name':'Decomposition level','indexing':'1 indexed'})
ds_T['j_low'] =  xr.DataArray(j_low,dims=['js'],attrs={'long_name':'Lower bound of passband'})
ds_T['j_high'] =  xr.DataArray(j_high,dims=['js'],attrs={'long_name':'Upper bound of passband'})

ds_T['wj'] =  xr.DataArray(np.asarray(T_wavelet_coeff),dims=['depth','js','time'],attrs={'long_name':'Temperature Wavelet Coefficients'})
ds_T['T_smooths'] =  xr.DataArray(np.asarray(T_smooth)[:,:len(ds_T.time.values)],dims=['depth','time'],attrs={'long_name':'Temperature Smooths'})
ds_T['T_details'] =  xr.DataArray(np.asarray(T_details)[:,:,:len(ds_T.time.values)],dims=['depth','js','time'],attrs={'long_name':'Temperature Details'})

In [ ]:
# Calculate mean T at 1 min and longer scales
T_tide =  ds_T['Temperature'].rolling(time=3).mean()
dT_dz_tide =  np.abs(T_tide.differentiate('depth'))
###
rho_bkd =  slope*T_tide + intercept
drhobkd_dz =  rho_bkd.differentiate('depth')

### Calculate minimum buoyancy period
ds_T['N'] =  (np.sqrt(-9.81*drhobkd_dz/rho_bkd)).where(dT_dz_tide>0.001)
ds_T['bp'] =  2*np.pi/(60*ds_T['N'])
ds_T['mbp'] = ds_T['bp'].rolling(time=int(30*3),center=True,min_periods=1).min()
ds_T['mbp'] =  ds_T['mbp'].interpolate_na(dim='time',limit=120*3)

### Estimate temperature variance from time scales shorter than min. buoy per
ds_T['mask'] =  (ds_T['mbp'] > ds_T['j_high'])
ds_T['Tvar'] =  (ds_T['wj']**2).where(ds_T['mask']).sum(dim='js')

## Testing period
ds_T['T_low'] = ds_T['Temperature'].rolling(time=15,min_periods=1).mean()

## Calcualte background temperature gradient and corresponding value for N
ts =  np.empty_like(ds_T['T_low'].values)
dtsdz = np.empty_like(ds_T['T_low'].values)
rho_bkd = np.empty_like(ds_T['T_low'].values)
drhobkd_dz =  np.empty_like(ds_T['T_low'].values)
cheby_order_Temp = 10
for t,time in enumerate(ds_T['T_low'].time.values):
    subt = ds_T['T_low'].isel(time=t)
    Ts = subt.values[~np.isnan(subt.values)]
    Rs = slope*Ts+intercept
    
    Ds = subt.depth.values[~np.isnan(subt.values)]
    
    t_fit = np.polynomial.chebyshev.Chebyshev.fit(Ds,Ts,cheby_order_Temp)
    ts[:,t] = t_fit(subt.depth.values)
    dtsdz[:,t] = t_fit.deriv(m=1)(subt.depth.values)
    
    r_fit = np.polynomial.chebyshev.Chebyshev.fit(Ds,Rs,cheby_order_Temp)
    rho_bkd[:,t] = r_fit(subt.depth.values)
    drhobkd_dz[:,t] = r_fit.deriv(m=1)(subt.depth.values)
                                  
rho_bkd = xr.DataArray(rho_bkd,dims=['depth','time'],coords={'depth':ds_T.depth.values,'time':ds_T.time.values})
drhobkd_dz = rho_bkd.differentiate('depth')
###
Nb = (np.sqrt(-9.81*drhobkd_dz/rho_bkd))
ds_T['Nb'] = Nb 
ds_T['dTbdz'] = xr.DataArray(np.abs(dtsdz),dims=['depth','time'],coords={'depth':ds_T.depth.values,'time':ds_T.time.values})

### Shear

## Calculate what is needed for model
ds_T['Le'] = np.sqrt(ds_T['Tvar'])/ds_T['dTbdz']
ds_T['K'] =  0.09*(ds_T['Le']**2)*ds_T['Sb']
ds_T['Ri'] =  (ds_T['Nb']**2)/(ds_T['Sb']**2)
ds_T['JQ'] =  1025*4000*ds_T['K']*ds_T['dTbdz']

In [ ]:
### stationarity testing
dT = ds_T['Temperature'].diff(dim='time',n=1)
ready = dT.coarsen(time=3,boundary='pad').reduce(prep)
stat = ready.coarsen(time=10,boundary='pad').reduce(runs)
ds_T['stat'] = stat.reindex_like(ds_T['Temperature'],method='nearest')

In [ ]:
no = 3
mask = np.ones_like(ds_T['K'].values).astype('bool')
## Coarsen data from 20 sec sampling rate to 1 minute and QAQC for stationarity, minimum shear and temperature gradient 
ds_fs_coarse = xr.Dataset()
fn  =  geometric_mean_fn #mean_fn #
ds_fs_coarse['Temperature'] = ds_T['Temperature'].coarsen(time=no,boundary='pad').reduce(mean_fn)

mask *= (ds_T['dTbdz'] > 0.002)
ds_fs_coarse['dTbdz'] =  ds_T['dTbdz'].where(mask).coarsen(time=no,boundary='pad').reduce(fn)
ds_fs_coarse['Nb'] =  ds_T['Nb'].where(mask).coarsen(time=no,boundary='pad').reduce(fn)
ds_fs_coarse['bp'] =  ds_T['bp'].where(mask).coarsen(time=no,boundary='pad').reduce(mean_fn)
ds_fs_coarse['mbp'] =  ds_T['mbp'].where(mask).coarsen(time=no,boundary='pad').reduce(mean_fn)


mask = np.ones_like(ds_T['K'].values).astype('bool')
mask *= (ds_T['Sb'] > 0.004)
ds_fs_coarse['Sb'] =  ds_T['Sb'].where(mask).coarsen(time=no,boundary='pad').reduce(fn)

mask = np.ones_like(ds_T['K'].values).astype('bool')
mask *= (ds_T['Sb'] > 0.004)
mask *= (ds_T['dTbdz'] > 0.002)
ds_fs_coarse['Ri'] =  ds_T['Ri'].where(mask).coarsen(time=no,boundary='pad').reduce(fn)

mask *= (ds_T['K'] < 1)&(ds_T['K'] > 10**-7)
mask *= (ds_T['stat'] < 3)
ds_fs_coarse['Le'] =  ds_T['Le'].where(mask).coarsen(time=no,boundary='pad').reduce(fn)
ds_fs_coarse['Tvar'] =  ds_T['Tvar'].where(mask).coarsen(time=no,boundary='pad').reduce(fn)
ds_fs_coarse['K'] =  ds_T['K'].where(mask).coarsen(time=no,boundary='pad').reduce(fn) # mean_fn
ds_fs_coarse['JQ'] =  ds_T['JQ'].where(mask).coarsen(time=no,boundary='pad').reduce(fn) # geometric_mean_fn

ds_fs_coarse.to_netcdf(r'RS2019_T200_FS_20sec Shear' + str(cheby_order_shear)+' Temp'+str(cheby_order_Temp)+'.nc')